# ReadMe:  (basic introduction of our code)

* The actual dataset that I used is from kaggle : https://www.kaggle.com/datasets/jessicali9530/stanford-dogs-dataset

* It has 120 different folders for each kind of dog, with 20k images in total.

* We did data-cleaining as in deleting the folders of the dog-breeds which I did not need. So final number of folders was 40. With almost 7k+ something total images.

* We then converted the annotations into yolov5 format. The total number of images then reduced to 6876.

* Then we splitted our dataset into train, validation and test.

* We then chunked out train-data because it was too much for Colab to handle (code was crashing), into 5 chunks, each of almost 1100 images approximately.

* For training, I trained the 1st chunk, it's best weight was then used for the training of the 2nd chunk, and the same for repeated for the other 2 remaining chunks.

* The final best weight (of chunk4) was used for deploying the model.



# 1. Preparing data:

* upload the fypdataset.zip and annotations.zip file

In [ ]:
!unzip /content/fypdataset.zip

* functions to convert annotation files from xml to txt (yolov5 version)

In [ ]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

dirs = ['fypdataset']
classes = [
           "Chihuahua", "toy_terrier", "Rhodesian_ridgeback", "basset", "beagle", "bloodhound", "bluetick", "black-and-tan_coonhound", "Walker_hound", "English_foxhound", "redbone", "Italian_greyhound", "whippet", "Ibizan_hound", "Saluki", "Weimaraner", "Staffordshire_bullterrier", "American_Staffordshire_terrier", "golden_retriever", "Labrador_retriever", "Chesapeake_Bay_retriever", "German_short-haired_pointer", "Brittany_spaniel", "kuvasz", "schipperke", "malinois", "kelpie", "Rottweiler", "miniature_pinscher", "EntleBucher", "boxer", "Great_Dane", "Saint_Bernard", "Eskimo_dog", "basenji", "Leonberg", "Pembroke", "Cardigan", "Mexican_hairless", "dingo", "African_hunting_dog"       ]

def getImagesInDir(dir_path):
    image_list = []
    for filename in os.listdir(dir_path+"/images"):
      image_list.append(filename)

    return image_list

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]
    # image_path is like n02085620_10074.jpg
    # basename_no_ext is like n02085620_10074
    # dir_path is /content/fypdataset/
    # output_path is /content/fypdataset/yolo
    # in_file should be: /content/fypdataset/labels/n02085620_10074
    in_file = open(dir_path + '/labels/' + basename_no_ext)
    out_file = open(output_path + "/" + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = 0
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
             float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

cwd = getcwd()

for dir_path in dirs:
    full_dir_path = cwd + '/' + dir_path
    print(full_dir_path)
    output_path = full_dir_path +'/yolo'
    print(output_path)

    if not os.path.exists(output_path):
        os.makedirs(output_path)
        print("output path created!")

    image_paths = getImagesInDir(full_dir_path)
    list_file = open(full_dir_path + '.txt', 'w')

    for image_path in image_paths:
        list_file.write(image_path + '\n')
        convert_annotation(full_dir_path, output_path, image_path)
    list_file.close()

    print("Finished processing: " + dir_path)

/content/fypdataset
/content/fypdataset/yolo
Finished processing: fypdataset


* train, val, test split:

In [ ]:
# code to confirm my logic XD

list=[1,2,3]

for i in list:
  print(i)

1
2
3


In [ ]:
from sklearn.model_selection import train_test_split
import re
# split the image data into train, val, and test sets

path_to_annotation_files = []
path_to_image_files = []
annotation_root= "/content/fypdataset/yolo"
root_path = "/content/fypdataset"
for class_id in os.listdir(annotation_root):
  annotation_path = os.path.join(annotation_root, class_id)       #/content/fypdataset/yolo/n02085620_1298.txt
  text=class_id                         # we need i == n02110806_3971.jpg
  d=re.search ('\d+_\d+', text)
  if d.group(0)!=None:
    b=d.group(0)
    x="n"+b+".jpg"     #x= n02085620_1298.jpg
  image_path = os.path.join(root_path, "images/"+x)   #/content/fypdataset/images/n02085620_1298.jpg
  path_to_annotation_files.append(annotation_path)
  path_to_image_files.append(image_path)



print("no of images: ", len(path_to_annotation_files))
print("no of labels: ", len(path_to_image_files))

train_images, val_images, train_annotations, val_annotations= train_test_split(path_to_image_files, path_to_annotation_files,
                                                                               test_size=0.2, random_state=1)

val_images, test_images, val_annotations, test_annotations=train_test_split(val_images, val_annotations,
                                                                           test_size=0.5, random_state=1)



no of images:  6876
no of labels:  6876


In [ ]:
print(train_images) #/content/fypdataset/images/n02085620_10074.jpg
print(train_annotations) #/content/fypdataset/yolo/n02085620_1073.txt

['/content/fypdataset/images/n02088466_4169.jpg', '/content/fypdataset/images/n02108089_9778.jpg', '/content/fypdataset/images/n02088238_8520.jpg', '/content/fypdataset/images/n02101388_9999.jpg', '/content/fypdataset/images/n02105412_2803.jpg', '/content/fypdataset/images/n02099712_7605.jpg', '/content/fypdataset/images/n02107312_1586.jpg', '/content/fypdataset/images/n02085620_1205.jpg', '/content/fypdataset/images/n02099849_3617.jpg', '/content/fypdataset/images/n02111129_2750.jpg', '/content/fypdataset/images/n02105162_6301.jpg', '/content/fypdataset/images/n02113186_6992.jpg', '/content/fypdataset/images/n02092339_5999.jpg', '/content/fypdataset/images/n02088238_9626.jpg', '/content/fypdataset/images/n02093428_16887.jpg', '/content/fypdataset/images/n02108000_710.jpg', '/content/fypdataset/images/n02093256_2513.jpg', '/content/fypdataset/images/n02104029_2992.jpg', '/content/fypdataset/images/n02099712_7802.jpg', '/content/fypdataset/images/n02091244_873.jpg', '/content/fypdataset

# moving the train, test and val split files into their respective folders:

* I created 2 new folders as images1 and labels1 in fypdataset folder, and then I created training, testing and validation folders within each of them. So that I can delete the old images and yolo folder (I also deleted the initial labels/annotations folder)

In [ ]:
import shutil

def moves_files_to_folder(list_of_files, destination_folder):
  for f in list_of_files:
    try:
      shutil.copy(f, destination_folder)
    except:
      print(f)
      assert False

moves_files_to_folder(train_images, "/content/fypdataset/images1/training")
moves_files_to_folder(val_images, "/content/fypdataset/images1/validation")
moves_files_to_folder(test_images, "/content/fypdataset/images1/testing")
moves_files_to_folder(train_annotations, "/content/fypdataset/labels1/training")
moves_files_to_folder(val_annotations, "/content/fypdataset/labels1/validation")
moves_files_to_folder(test_annotations, "/content/fypdataset/labels1/testing")



# Now deleting the old images and yolo folder:

In [ ]:
shutil.rmtree("/content/fypdataset/images")
shutil.rmtree("/content/fypdataset/yolo")

# chunking data into multiple training files for comparatively quick training:


* create 5 folders as training1, training2, training3, training4 and training5 in both, images and labels folder.

**NOTE: I chunked my data into multiple folders because back then, I did not know that I can just use FREE GPU and give the whole dataset for training in one go!**

In [ ]:
# importing dataset again: (if you reconnected your runtime/ reuploaded your fypdataset.zip)

!unzip /content/fypdataset.zip


In [ ]:
# for image training data:
import os

path= "/content/content/fypdataset/images/training"
j=0

chunki1=[]
chunki2=[]
chunki3=[]
chunki4=[]
chunki5=[]


for i in os.listdir(path):
  if j>=0 and j<1100:
    chunki1.append(i)

  if j>=1100 and j<2200:
    chunki2.append(i)

  if j>=2200 and j<3300:
    chunki3.append(i)

  if j>=3300 and j<4400:
    chunki4.append(i)

  if j>=4400 and j<5500:
    chunki5.append(i)
  j+=1




In [ ]:
# print("chunk1: \n", chunki1)
print("chunk2: \n", chunki2)
print("chunk3: \n", chunki3)
print("chunk4: \n", chunki4)
print("chunk5: \n", chunki5)

In [ ]:
# now divide the data into content/fypdataset/images/training1, training2, training3, training4
#newchunki1=[]
newchunki2=[]
newchunki3=[]
newchunki4=[]
newchunki5=[]

# for i in chunki1:
#   newchunki1.append("/content/fypdataset/images/training/"+i)

for i in chunki2:
  newchunki2.append("/content/content/fypdataset/images/training/"+i)

for i in chunki3:
  newchunki3.append("/content/content/fypdataset/images/training/"+i)

for i in chunki4:
  newchunki4.append("/content/content/fypdataset/images/training/"+i)

for i in chunki5:
  newchunki5.append("/content/content/fypdataset/images/training/"+i)


# moves_files_to_folder(newchunki1, "/content/fypdataset/images/training1")
moves_files_to_folder(newchunki2, "/content/content/fypdataset/images/training2")
moves_files_to_folder(newchunki3, "/content/content/fypdataset/images/training3")
moves_files_to_folder(newchunki4, "/content/content/fypdataset/images/training4")
moves_files_to_folder(newchunki5, "/content/content/fypdataset/images/training5")


In [ ]:
# for labels training data:


#chunkl1=[]
chunkl2=[]
chunkl3=[]
chunkl4=[]
chunkl5=[]

# for i in chunki1:
#   base = os.path.splitext(i)[0]
#   chunkl1.append(base+".txt")

for i in chunki2:
  base = os.path.splitext(i)[0]
  chunkl2.append(base+".txt")

for i in chunki3:
  base = os.path.splitext(i)[0]
  chunkl3.append(base+".txt")

for i in chunki4:
  base = os.path.splitext(i)[0]
  chunkl4.append(base+".txt")

for i in chunki5:
  base = os.path.splitext(i)[0]
  chunkl5.append(base+".txt")



In [ ]:
#print("chunk1: \n", chunkl1)
print("chunk2: \n", chunkl2)
print("chunk3: \n", chunkl3)
print("chunk4: \n", chunkl4)
print("chunk5: \n", chunkl5)

chunk2: 
 ['n02087394_3314.txt', 'n02116738_8738.txt', 'n02093256_5646.txt', 'n02088632_874.txt', 'n02089078_1681.txt', 'n02113023_436.txt', 'n02091831_12653.txt', 'n02089078_2957.txt', 'n02108089_959.txt', 'n02104029_3787.txt', 'n02104365_740.txt', 'n02107312_4996.txt', 'n02108000_1600.txt', 'n02088466_7178.txt', 'n02099712_4550.txt', 'n02113978_1823.txt', 'n02091032_11998.txt', 'n02108000_3002.txt', 'n02088632_4584.txt', 'n02109047_20229.txt', 'n02110806_757.txt', 'n02088466_7269.txt', 'n02106550_7767.txt', 'n02090379_1964.txt', 'n02115641_6183.txt', 'n02091032_4012.txt', 'n02088466_5784.txt', 'n02100236_1652.txt', 'n02093256_911.txt', 'n02091831_4259.txt', 'n02091134_2349.txt', 'n02099849_2646.txt', 'n02091244_5637.txt', 'n02089078_2025.txt', 'n02088466_1671.txt', 'n02099849_4413.txt', 'n02105412_2085.txt', 'n02107312_5665.txt', 'n02093256_6177.txt', 'n02088466_2934.txt', 'n02088364_4493.txt', 'n02113978_2337.txt', 'n02116738_3024.txt', 'n02106550_148.txt', 'n02104365_6575.txt', 'n0

In [ ]:
# now divide the data into content/fypdataset/labels/training1, training2, training3, training4

#newchunkl1=[]
newchunkl2=[]
newchunkl3=[]
newchunkl4=[]
newchunkl5=[]

# for i in chunkl1:
#   newchunkl1.append("/content/fypdataset/labels/training/"+i)


for i in chunkl2:
  newchunkl2.append("/content/content/fypdataset/labels/training/"+i)


for i in chunkl3:
  newchunkl3.append("/content/content/fypdataset/labels/training/"+i)


for i in chunkl4:
  newchunkl4.append("/content/content/fypdataset/labels/training/"+i)


for i in chunkl5:
  newchunkl5.append("/content/content/fypdataset/labels/training/"+i)


# moves_files_to_folder(newchunkl1, "/content/fypdataset/labels/training1")
moves_files_to_folder(newchunkl2, "/content/content/fypdataset/labels/training2")
moves_files_to_folder(newchunkl3, "/content/content/fypdataset/labels/training3")
moves_files_to_folder(newchunkl4, "/content/content/fypdataset/labels/training4")
moves_files_to_folder(newchunkl5, "/content/content/fypdataset/labels/training5")




# now deleting the training folder from images and labels folder:


In [ ]:
shutil.rmtree("/content/content/fypdataset/images/training")
shutil.rmtree("/content/content/fypdataset/images/training")

# converting fypdataset into zipped file,, and downloading it, so that we dont lose our data.


In [ ]:

!zip /content/content/fypdataset.zip -r /content/content/fypdataset
# from google.colab import files
# files.download("/content/fypdataset.zip")

# you can also just double-click on the zip file as shown in the files tab

# 2. Working on the object detection model :

In [ ]:
# incase you had to reconnect the runtime, or you reuploaded the zip file
!unzip /content/fypdataset.zip

* cloning with yolov5 repository

In [ ]:
!git clone https://github.com/ultralytics/yolov5


Cloning into 'yolov5'...
remote: Enumerating objects: 12773, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 12773 (delta 29), reused 40 (delta 22), pack-reused 12719
Receiving objects: 100% (12773/12773), 12.34 MiB | 13.78 MiB/s, done.
Resolving deltas: 100% (8786/8786), done.


* changing directory to yolov5

In [ ]:
%cd yolov5

/content/yolov5


* installing requirements for yolov5

In [ ]:
%pip install -r requirements.txt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 4.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# training our model :

1. we will first train our chunk 1, and it's output best weight will be input to the weights for training chunk2, and then its output best weight will be input to chunk3, and the same thing will be followed for chunk4.


* Training chunk1:

In [ ]:
!python train.py --img 640 --batch 12 --epochs 10  --data /content/yolov5/data/custom_dataset.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data/custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=12, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-304-g51fb467 Python-3.7.13 torch-1.11.0+cu102 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --data /content/yolov5/data/custom_dataset.yaml --img 640
# for testing

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --img 640 --conf 0.5 --source "/content/fypdataset/images/testing"
# for inference
# detecting new images

# which folders to delete:

In [ ]:
import shutil
shutil.rmtree("/content/yolov5/runs/train/exp")
#delete exp 3 4 and 5 from train
# exp2 from detect
# exp, exp2 and exp4 from val

downloaded the files to continue work later on...

In [ ]:
# !zip -r /content/fypdataset.zip /content/fypdataset

In [ ]:
!zip -r /content/yolov5.zip /content/yolov5

In [ ]:
# from google.colab import files
# files.download("/content/fypdataset.zip")
# files.download("/content/fypdataset(chunked)")

Upload:

1. Fypdataset (the chunked one)
2. best.pt (of trained chunk1) ----- [to train chunk2]

* you'll have to create the custom_dataset.yaml file in the data folder of yolov5, everytime when you upload your dataset and when you clone the yolov5 repository :))


In [ ]:
!unzip /content/fypdataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: fypdataset/labels/training2/n02088238_11917.txt  
  inflating: fypdataset/labels/training2/n02088238_12966.txt  
  inflating: fypdataset/labels/training2/n02088238_13036.txt  
  inflating: fypdataset/labels/training2/n02088238_13222.txt  
  inflating: fypdataset/labels/training2/n02088238_13683.txt  
  inflating: fypdataset/labels/training2/n02088238_3475.txt  
  inflating: fypdataset/labels/training2/n02088238_6017.txt  
  inflating: fypdataset/labels/training2/n02088238_7232.txt  
  inflating: fypdataset/labels/training2/n02088238_9015.txt  
  inflating: fypdataset/labels/training2/n02088238_9162.txt  
  inflating: fypdataset/labels/training2/n02088238_9585.txt  
  inflating: fypdataset/labels/training2/n02088238_9626.txt  
  inflating: fypdataset/labels/training2/n02088364_10206.txt  
  inflating: fypdataset/labels/training2/n02088364_10575.txt  
  inflating: fypdataset/labels/training2/n02088364_10731.txt  
  inflating

* cloning yolov5:

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 12805, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 12805 (delta 48), reused 69 (delta 40), pack-reused 12719
Receiving objects: 100% (12805/12805), 12.40 MiB | 26.40 MiB/s, done.
Resolving deltas: 100% (8803/8803), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 5.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


* Training chunk2:

--> Update the train file path in the custom_dataset.yaml file for the 2nd chunk

In [ ]:
!python train.py --img 640 --batch 12 --epochs 10  --data /content/yolov5/data/custom_dataset.yaml --weights /content/best.pt --cache

train: weights=/content/best.pt, cfg=, data=/content/yolov5/data/custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=12, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.7.13 torch-1.12.0+cu113 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, h

In [ ]:
!python val.py --weights /content/best.pt --data /content/yolov5/data/custom_dataset.yaml --img 640


val: data=/content/yolov5/data/custom_dataset.yaml, weights=['/content/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.7.13 torch-1.12.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning '/content/content/fypdataset/labels/validation.cache' images and labels... 688 found, 0 missing, 0 empty, 0 corrupt: 100% 688/688 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 22/22 [03:31<00:00,  9.64s/it]
                 all        688        749      0.914      0.924      0.941      0.652
Speed: 2.3ms pre-process, 302.5ms inference, 0.5ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs/val/e

In [ ]:
!unzip /content/dogs.zip

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source "/content/yolov5/dogs"
# for inference
# detecting new images

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source "/content/content/fypdataset/images/testing"
# for inference
# detecting new images

In [ ]:
!zip /content/content.zip -r /content/content

In [ ]:
!zip /content/yolov5.zip -r /content/yolov5

* Training chunk3:

In [ ]:
!python train.py --img 640 --batch 12 --epochs 10  --data /content/yolov5/data/custom_dataset.yaml --weights /content/yolov5/runs/train/exp/weights/best.pt --cache

train: weights=/content/yolov5/runs/train/exp/weights/best.pt, cfg=, data=/content/yolov5/data/custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=12, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/ultralytics/yolov5
   2d34408..1ad285a  classifier -> origin/classifier
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 

In [ ]:
!python val.py --weights /content/best.pt --data /content/yolov5/custom_dataset.yaml --img 640


val: data=/content/yolov5/custom_dataset.yaml, weights=['/content/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.7.13 torch-1.12.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
100% 755k/755k [00:00<00:00, 14.6MB/s]
val: Scanning '/content/fypdataset/labels/validation' images and labels...688 found, 0 missing, 0 empty, 0 corrupt: 100% 688/688 [00:00<00:00, 1538.33it/s]
val: New cache created: /content/fypdataset/labels/validation.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 22/22 [04:11<00:00, 11.45s/it]
                 all        688        749      0.949      0.945      0.967      0.736
Speed: 2.5ms pre-proce

In [ ]:
!python detect.py --weights /content/best.pt --img 640 --conf 0.5 --source /content/fypdataset/images/testing


detect: weights=['/content/best.pt'], source=/content/fypdataset/images/testing, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.7.13 torch-1.12.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/688 /content/fypdataset/images/testing/n02085620_11477.jpg: 480x640 1 Dog, Done. (0.298s)
image 2/688 /content/fypdataset/images/testing/n02085620_12718.jpg: 640x448 1 Dog, Done. (0.281s)
image 3/688 /content/fypdataset/images/testing/n02085620_1298.jpg: 448x640 Done. (0.261s)
image 4/688 /content/fypdataset/images/testing/n02085620_1321.jpg: 448x640 1 Dog, Done. (0.254s)
i

In [ ]:
!zip /content/yolov5/runs/val.zip -r /content/yolov5/runs/val
!zip /content/yolov5/runs/detect.zip -r /content/yolov5/runs/detect

* Training chunk3:

In [ ]:
!python train.py --img 640 --batch 12 --epochs 10  --data /content/yolov5/data/custom_dataset.yaml --weights /content/best.pt --cache

train: weights=/content/best.pt, cfg=, data=/content/yolov5/data/custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=12, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 6 (delta 4), reused 6 (delta 4), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/ultralytics/yolov5
   c526341..62cceed  classifier -> origin/classifier
github: up to date with https://github.com/ultralytic

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data /content/yolov5/data/custom_dataset.yaml --img 640
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source /content/fypdataset/images/testing


val: data=/content/yolov5/data/custom_dataset.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.7.13 torch-1.12.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning '/content/fypdataset/labels/validation.cache' images and labels... 688 found, 0 missing, 0 empty, 0 corrupt: 100% 688/688 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 22/22 [04:20<00:00, 11.83s/it]
                 all        688        749      0.959      0.936       0.97      0.752
Speed: 2.9ms pre-process, 370.7ms inference, 0.7ms NMS per image at shape (32, 3, 640, 640)
Resul

In [ ]:
!zip /content/yolov5/runs/train/exp.zip -r /content/yolov5/runs/train/exp

  adding: content/yolov5/runs/train/exp/ (stored 0%)
  adding: content/yolov5/runs/train/exp/val_batch2_pred.jpg (deflated 7%)
  adding: content/yolov5/runs/train/exp/val_batch2_labels.jpg (deflated 7%)
  adding: content/yolov5/runs/train/exp/train_batch2.jpg (deflated 5%)
  adding: content/yolov5/runs/train/exp/train_batch0.jpg (deflated 5%)
  adding: content/yolov5/runs/train/exp/results.csv (deflated 82%)
  adding: content/yolov5/runs/train/exp/opt.yaml (deflated 43%)
  adding: content/yolov5/runs/train/exp/val_batch0_labels.jpg (deflated 9%)
  adding: content/yolov5/runs/train/exp/val_batch1_labels.jpg (deflated 7%)
  adding: content/yolov5/runs/train/exp/confusion_matrix.png (deflated 40%)
  adding: content/yolov5/runs/train/exp/labels_correlogram.jpg (deflated 30%)
  adding: content/yolov5/runs/train/exp/F1_curve.png (deflated 16%)
  adding: content/yolov5/runs/train/exp/results.png (deflated 7%)
  adding: content/yolov5/runs/train/exp/weights/ (stored 0%)
  adding: content/yolov

In [ ]:
!python detect.py --weights /content/best.pt --img 640 --conf 0.5 --source /content/video1.mp4

Streaming output truncated to the last 5000 lines.
video 1/1 (13166/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.333s)
video 1/1 (13167/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.405s)
video 1/1 (13168/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.402s)
video 1/1 (13169/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.395s)
video 1/1 (13170/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.385s)
video 1/1 (13171/18163) /content/video1.mp4: 384x640 Done. (0.373s)
video 1/1 (13172/18163) /content/video1.mp4: 384x640 Done. (0.235s)
video 1/1 (13173/18163) /content/video1.mp4: 384x640 Done. (0.235s)
video 1/1 (13174/18163) /content/video1.mp4: 384x640 Done. (0.233s)
video 1/1 (13175/18163) /content/video1.mp4: 384x640 Done. (0.246s)
video 1/1 (13176/18163) /content/video1.mp4: 384x640 Done. (0.234s)
video 1/1 (13177/18163) /content/video1.mp4: 384x640 Done. (0.226s)
video 1/1 (13178/18163) /content/video1.mp4: 384x640 Done. (0.225s)
video 1/1 (13179/18163) /conte

* Training chunk5:

In [ ]:
!python train.py --img 640 --batch 12 --epochs 10  --data /content/yolov5/data/custom_dataset.yaml --weights /content/best.pt --cache

train: weights=/content/best.pt, cfg=, data=/content/yolov5/data/custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=12, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.7.13 torch-1.12.0+cu113 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, h

In [ ]:
!zip /content/yolov5/runs/train/exp -r /content/yolov5/runs/train/exp

  adding: content/yolov5/runs/train/exp/ (stored 0%)
  adding: content/yolov5/runs/train/exp/val_batch2_labels.jpg (deflated 7%)
  adding: content/yolov5/runs/train/exp/P_curve.png (deflated 21%)
  adding: content/yolov5/runs/train/exp/opt.yaml (deflated 43%)
  adding: content/yolov5/runs/train/exp/R_curve.png (deflated 18%)
  adding: content/yolov5/runs/train/exp/F1_curve.png (deflated 17%)
  adding: content/yolov5/runs/train/exp/val_batch1_labels.jpg (deflated 7%)
  adding: content/yolov5/runs/train/exp/val_batch0_labels.jpg (deflated 9%)
  adding: content/yolov5/runs/train/exp/train_batch2.jpg (deflated 5%)
  adding: content/yolov5/runs/train/exp/val_batch1_pred.jpg (deflated 7%)
  adding: content/yolov5/runs/train/exp/events.out.tfevents.1658297741.300a0d7c335e.646.0 (deflated 39%)
  adding: content/yolov5/runs/train/exp/results.csv (deflated 82%)
  adding: content/yolov5/runs/train/exp/hyp.yaml (deflated 45%)
  adding: content/yolov5/runs/train/exp/results.png (deflated 7%)
  addi

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data /content/yolov5/data/custom_dataset.yaml --img 640
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source /content/fypdataset/images/testing


val: data=/content/yolov5/data/custom_dataset.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.7.13 torch-1.12.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning '/content/fypdataset/labels/validation.cache' images and labels... 688 found, 0 missing, 0 empty, 0 corrupt: 100% 688/688 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 22/22 [03:53<00:00, 10.61s/it]
                 all        688        749      0.946      0.951      0.974      0.759
Speed: 2.3ms pre-process, 333.8ms inference, 0.5ms NMS per image at shape (32, 3, 640, 640)
Resul

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source /content/video1.mp4
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source /content/video2.mp4

Streaming output truncated to the last 5000 lines.
video 1/1 (17897/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.232s)
video 1/1 (17898/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.227s)
video 1/1 (17899/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.229s)
video 1/1 (17900/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.246s)
video 1/1 (17901/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.228s)
video 1/1 (17902/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.224s)
video 1/1 (17903/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.236s)
video 1/1 (17904/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.240s)
video 1/1 (17905/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.224s)
video 1/1 (17906/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.238s)
video 1/1 (17907/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.226s)
video 1/1 (17908/18163) /content/video1.mp4: 384x640 1 Dog, Done. (0.246s)
video 1/1 (17909/18163) /content/video1.mp4: 384x

In [ ]:
!zip /content/yolov5/runs/val/exp -r /content/yolov5/runs/val/exp

  adding: content/yolov5/runs/val/exp/ (stored 0%)
  adding: content/yolov5/runs/val/exp/val_batch2_labels.jpg (deflated 7%)
  adding: content/yolov5/runs/val/exp/P_curve.png (deflated 21%)
  adding: content/yolov5/runs/val/exp/R_curve.png (deflated 18%)
  adding: content/yolov5/runs/val/exp/F1_curve.png (deflated 16%)
  adding: content/yolov5/runs/val/exp/val_batch1_labels.jpg (deflated 8%)
  adding: content/yolov5/runs/val/exp/val_batch0_labels.jpg (deflated 9%)
  adding: content/yolov5/runs/val/exp/val_batch1_pred.jpg (deflated 7%)
  adding: content/yolov5/runs/val/exp/val_batch2_pred.jpg (deflated 7%)
  adding: content/yolov5/runs/val/exp/PR_curve.png (deflated 25%)
  adding: content/yolov5/runs/val/exp/val_batch0_pred.jpg (deflated 9%)
  adding: content/yolov5/runs/val/exp/confusion_matrix.png (deflated 40%)


In [ ]:
!zip /content/yolov5/runs/detect/exp5 -r /content/yolov5/runs/detect/exp5

  adding: content/yolov5/runs/detect/exp5/ (stored 0%)
  adding: content/yolov5/runs/detect/exp5/n02087394_889.jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp5/n02108089_2670.jpg (deflated 4%)
  adding: content/yolov5/runs/detect/exp5/n02088238_5441.jpg (deflated 4%)
  adding: content/yolov5/runs/detect/exp5/n02108089_1104.jpg (deflated 4%)
  adding: content/yolov5/runs/detect/exp5/n02115641_9675.jpg (deflated 4%)
  adding: content/yolov5/runs/detect/exp5/n02105162_6106.jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp5/n02105162_5927.jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp5/n02089973_1356.jpg (deflated 2%)
  adding: content/yolov5/runs/detect/exp5/n02087046_6179.jpg (deflated 3%)
  adding: content/yolov5/runs/detect/exp5/n02087046_4751.jpg (deflated 4%)
  adding: content/yolov5/runs/detect/exp5/n02113978_3794.jpg (deflated 4%)
  adding: content/yolov5/runs/detect/exp5/n02099849_3812.jpg (deflated 6%)
  adding: content/yolov5/runs/detect/exp5/n021

In [ ]:
!unzip /content/dogs.zip

Archive:  /content/dogs.zip
  inflating: dogs/5d8fa8bb0c0b0.jpg  
  inflating: dogs/download (1).jpg   
  inflating: dogs/download (10).jpg  
  inflating: dogs/download (11).jpg  
  inflating: dogs/download (12).jpg  
  inflating: dogs/download (13).jpg  
  inflating: dogs/download (14).jpg  
  inflating: dogs/download (15).jpg  
  inflating: dogs/download (16).jpg  
  inflating: dogs/download (17).jpg  
  inflating: dogs/download (18).jpg  
  inflating: dogs/download (19).jpg  
  inflating: dogs/download (2).jpg   
  inflating: dogs/download (20).jpg  
  inflating: dogs/download (21).jpg  
  inflating: dogs/download (22).jpg  
  inflating: dogs/download (23).jpg  
  inflating: dogs/download (24).jpg  
  inflating: dogs/download (25).jpg  
  inflating: dogs/download (26).jpg  
  inflating: dogs/download (27).jpg  
  inflating: dogs/download (28).jpg  
  inflating: dogs/download (29).jpg  
  inflating: dogs/download (3).jpg   
  inflating: dogs/download (30).jpg  
  inflating: dogs/down

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.9 --source /content/yolov5/dogs


detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/yolov5/dogs, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.9, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.7.13 torch-1.12.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/180 /content/yolov5/dogs/5d8fa8bb0c0b0.jpg: 288x640 Done. (0.186s)
image 2/180 /content/yolov5/dogs/download (1).jpg: 448x640 Done. (0.292s)
image 3/180 /content/yolov5/dogs/download (10).jpg: 352x640 Done. (0.219s)
image 4/180 /content/yolov5/dogs/download (11).jpg: 416x640 Done. (0.266s)
image 5/180 /content/yolov5/dogs/download (12).jpg: 384x640 Done. (0.

In [ ]:
!zip /content/yolov5/runs/detect/exp8 -r /content/yolov5/runs/detect/exp8

  adding: content/yolov5/runs/detect/exp8/ (stored 0%)
  adding: content/yolov5/runs/detect/exp8/images - 2022-04-24T173043.365.jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp8/images (59).jpg (deflated 6%)
  adding: content/yolov5/runs/detect/exp8/images (71).jpg (deflated 6%)
  adding: content/yolov5/runs/detect/exp8/images (27).jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp8/images (66).jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp8/images - 2022-04-24T173610.609.jpg (deflated 9%)
  adding: content/yolov5/runs/detect/exp8/images (88).jpg (deflated 11%)
  adding: content/yolov5/runs/detect/exp8/images (98).jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp8/images (73).jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp8/download (9).jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp8/download (24).jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp8/images - 2022-04-24T173624.282.jpg (deflated 9%)
  adding: content/yo

In [ ]:
!python detect.py --weights /content/best.pt --img 640 --conf 0.9 --source /content/yolov5/dogs


detect: weights=['/content/best.pt'], source=/content/yolov5/dogs, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.9, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.7.13 torch-1.12.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/180 /content/yolov5/dogs/5d8fa8bb0c0b0.jpg: 288x640 Done. (0.199s)
image 2/180 /content/yolov5/dogs/download (1).jpg: 448x640 Done. (0.287s)
image 3/180 /content/yolov5/dogs/download (10).jpg: 352x640 Done. (0.236s)
image 4/180 /content/yolov5/dogs/download (11).jpg: 416x640 Done. (0.267s)
image 5/180 /content/yolov5/dogs/download (12).jpg: 384x640 Done. (0.252s)
image 6/180 /content/yol

In [ ]:
!zip /content/yolov5/runs/detect/exp9 -r /content/yolov5/runs/detect/exp9

  adding: content/yolov5/runs/detect/exp9/ (stored 0%)
  adding: content/yolov5/runs/detect/exp9/images - 2022-04-24T173043.365.jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp9/images (59).jpg (deflated 6%)
  adding: content/yolov5/runs/detect/exp9/images (71).jpg (deflated 6%)
  adding: content/yolov5/runs/detect/exp9/images (27).jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp9/images (66).jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp9/images - 2022-04-24T173610.609.jpg (deflated 9%)
  adding: content/yolov5/runs/detect/exp9/images (88).jpg (deflated 11%)
  adding: content/yolov5/runs/detect/exp9/images (98).jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp9/images (73).jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp9/download (9).jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp9/download (24).jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp9/images - 2022-04-24T173624.282.jpg (deflated 9%)
  adding: content/yo

In [ ]:
# until chunk 4 was trained:

!python detect.py --weights /content/best.pt --img 640 --conf 0.5 --source /content/yolov5/dogs

detect: weights=['/content/best.pt'], source=/content/yolov5/dogs, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.7.13 torch-1.12.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/180 /content/yolov5/dogs/5d8fa8bb0c0b0.jpg: 288x640 5 Dogs, Done. (0.195s)
image 2/180 /content/yolov5/dogs/download (1).jpg: 448x640 3 Dogs, Done. (0.302s)
image 3/180 /content/yolov5/dogs/download (10).jpg: 352x640 3 Dogs, Done. (0.232s)
image 4/180 /content/yolov5/dogs/download (11).jpg: 416x640 4 Dogs, Done. (0.263s)
image 5/180 /content/yolov5/dogs/download (12).jpg: 384x640 4 Dogs,

In [ ]:
#until chunk 5 was trained:

!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source /content/yolov5/dogs

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/yolov5/dogs, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.7.13 torch-1.12.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/180 /content/yolov5/dogs/5d8fa8bb0c0b0.jpg: 288x640 5 Dogs, Done. (0.209s)
image 2/180 /content/yolov5/dogs/download (1).jpg: 448x640 2 Dogs, Done. (0.297s)
image 3/180 /content/yolov5/dogs/download (10).jpg: 352x640 2 Dogs, Done. (0.242s)
image 4/180 /content/yolov5/dogs/download (11).jpg: 416x640 4 Dogs, Done. (0.268s)
image 5/180 /content/yolov5/dogs/down

exp13 --> chunk5
exp12 --> chunk4

In [ ]:

!zip /content/yolov5/runs/detect/exp13 -r /content/yolov5/runs/detect/exp13

  adding: content/yolov5/runs/detect/exp13/ (stored 0%)
  adding: content/yolov5/runs/detect/exp13/images - 2022-04-24T173043.365.jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp13/images (59).jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp13/images (71).jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp13/images (27).jpg (deflated 3%)
  adding: content/yolov5/runs/detect/exp13/images (66).jpg (deflated 4%)
  adding: content/yolov5/runs/detect/exp13/images - 2022-04-24T173610.609.jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp13/images (88).jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp13/images (98).jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp13/images (73).jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp13/download (9).jpg (deflated 3%)
  adding: content/yolov5/runs/detect/exp13/download (24).jpg (deflated 3%)
  adding: content/yolov5/runs/detect/exp13/images - 2022-04-24T173624.282.jpg (deflated 9%)
  adding

* ------------- the weights of chunk5 are the best! :))


# working on fypdataset3

In [ ]:
!unzip /content/fypdataset3.zip

* we got some png images, so changing them into jpg:

In [ ]:
import os
from pathlib import Path

image_path = "/content/fypdataset3/images"

for i in os.listdir(image_path):
    base = os.path.splitext(i)[0]
    os.rename("/content/fypdataset3/images/"+i, "/content/fypdataset3/images/" +base + '.jpg')


* train, val, test split:

In [ ]:
from sklearn.model_selection import train_test_split
import re

annotation_root="/content/fypdataset3/labels"
root_path="/content/fypdataset3"

data = {}
path_to_annotation_files = []
path_to_image_files = []
for i in os.listdir(annotation_root):
  annotation_path = os.path.join(annotation_root, i)
  y= re.split(".txt",i)
  image_path = os.path.join(root_path, f"images/{y[0]}.jpg")
  path_to_annotation_files.append(annotation_path)
  path_to_image_files.append(image_path)


print("no of images: ", len(path_to_annotation_files))
print("no of labels: ", len(path_to_image_files))

train_images, val_images, train_annotations, val_annotations= train_test_split(path_to_image_files, path_to_annotation_files,
                                                                               test_size=0.2, random_state=1)

val_images, test_images, val_annotations, test_annotations=train_test_split(val_images, val_annotations,
                                                                           test_size=0.5, random_state=1)



In [ ]:
import shutil

def moves_files_to_folder(list_of_files, destination_folder):
  for f in list_of_files:
    try:
      shutil.copy(f, destination_folder)
    except:
      print(f)
      assert False

moves_files_to_folder(train_images, "/content/fypdataset3/images/training")
moves_files_to_folder(val_images, "/content/fypdataset3/images/validation")
moves_files_to_folder(test_images, "/content/fypdataset3/images/testing")
moves_files_to_folder(train_annotations, "/content/fypdataset3/labels/training")
moves_files_to_folder(val_annotations, "/content/fypdataset3/labels/validation")
moves_files_to_folder(test_annotations, "/content/fypdataset3/labels/testing")



In [ ]:
import shutil
shutil.rmtree("/content/fypdataset3/labels1")
shutil.rmtree("/content/fypdataset3/images1")

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

* train, detect and validate your data :))

In [ ]:
!python train.py --img 640 --batch 12 --epochs 10  --data /content/yolov5/data/custom_dataset.yaml --weights /content/best.pt --cache

# 100th epoch:

In [ ]:
!unzip /content/drive/MyDrive/fyp/fypdataset.zip
!unzip /content/drive/MyDrive/fyp/fypdataset2.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/fypdataset/images/training/n02104365_1158.jpg  
  inflating: content/fypdataset/images/training/human01-21.jpg  
  inflating: content/fypdataset/images/training/n02088466_5784.jpg  
  inflating: content/fypdataset/images/training/horse24-2.jpg  
  inflating: content/fypdataset/images/training/n02107312_1233.jpg  
  inflating: content/fypdataset/images/training/n02113186_9924.jpg  
  inflating: content/fypdataset/images/training/n02113978_1773.jpg  
  inflating: content/fypdataset/images/training/n02109525_243.jpg  
  inflating: content/fypdataset/images/training/n02104365_2424.jpg  
  inflating: content/fypdataset/images/training/n02090379_5005.jpg  
  inflating: content/fypdataset/images/training/n02087394_681.jpg  
  inflating: content/fypdataset/images/training/n02115641_3214.jpg  
  inflating: content/fypdataset/images/training/n02099601_176.jpg  
  inflating: content/fypdataset/images/training/n02093428_1767.j

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 13289, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 13289 (delta 53), reused 72 (delta 34), pack-reused 13188
Receiving objects: 100% (13289/13289), 12.57 MiB | 23.93 MiB/s, done.
Resolving deltas: 100% (9108/9108), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 4.8 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import subprocess
subprocess.call(["/content/yolov5/data/scripts/download_weights.sh"])

0

In [ ]:
!python train.py --img 640 --cfg /content/yolov5/models/yolov5m.yaml --hyp /content/yolov5/data/hyps/hyp.scratch-med.yaml --batch 32 --epochs 10 --data /content/yolov5/data/custom_dataset.yaml --weights /content/drive/MyDrive/FYP/exp/content/yolov5/runs/train/exp3/weights/best.pt --workers 24

train: weights=/content/drive/MyDrive/FYP/exp/content/yolov5/runs/train/exp3/weights/best.pt, cfg=/content/yolov5/models/yolov5m.yaml, data=/content/yolov5/data/custom_dataset.yaml, hyp=/content/yolov5/data/hyps/hyp.scratch-med.yaml, epochs=10, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.1-344-g0e165c5 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epo

In [ ]:
!zip /content/exp4 -r /content/yolov5/runs/train/exp4




# 120 epochs,, haha

  adding: content/yolov5/runs/train/exp4/ (stored 0%)
  adding: content/yolov5/runs/train/exp4/labels.jpg (deflated 17%)
  adding: content/yolov5/runs/train/exp4/labels_correlogram.jpg (deflated 20%)
  adding: content/yolov5/runs/train/exp4/val_batch2_pred.jpg (deflated 6%)
  adding: content/yolov5/runs/train/exp4/confusion_matrix.png (deflated 31%)
  adding: content/yolov5/runs/train/exp4/hyp.yaml (deflated 43%)
  adding: content/yolov5/runs/train/exp4/opt.yaml (deflated 46%)
  adding: content/yolov5/runs/train/exp4/val_batch0_pred.jpg (deflated 6%)
  adding: content/yolov5/runs/train/exp4/val_batch2_labels.jpg (deflated 6%)
  adding: content/yolov5/runs/train/exp4/events.out.tfevents.1659306015.84c201bac757.669.0 (deflated 36%)
  adding: content/yolov5/runs/train/exp4/weights/ (stored 0%)
  adding: content/yolov5/runs/train/exp4/weights/best.pt (deflated 8%)
  adding: content/yolov5/runs/train/exp4/weights/last.pt (deflated 8%)
  adding: content/yolov5/runs/train/exp4/train_batch0.jp

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp4/weights/best.pt --img 640 --conf 0.5 --source /content/drive/MyDrive/FYP/test_videos/cat_dog.mp4

Streaming output truncated to the last 5000 lines.
video 1/1 (13166/18163) /content/drive/.shortcut-targets-by-id/1WsrqcNndN0mishZGJ-SYTtzbRoQ2BzOQ/FYP/test_videos/cat_dog.mp4: 384x640 Done. (0.016s)
video 1/1 (13167/18163) /content/drive/.shortcut-targets-by-id/1WsrqcNndN0mishZGJ-SYTtzbRoQ2BzOQ/FYP/test_videos/cat_dog.mp4: 384x640 Done. (0.017s)
video 1/1 (13168/18163) /content/drive/.shortcut-targets-by-id/1WsrqcNndN0mishZGJ-SYTtzbRoQ2BzOQ/FYP/test_videos/cat_dog.mp4: 384x640 Done. (0.016s)
video 1/1 (13169/18163) /content/drive/.shortcut-targets-by-id/1WsrqcNndN0mishZGJ-SYTtzbRoQ2BzOQ/FYP/test_videos/cat_dog.mp4: 384x640 Done. (0.016s)
video 1/1 (13170/18163) /content/drive/.shortcut-targets-by-id/1WsrqcNndN0mishZGJ-SYTtzbRoQ2BzOQ/FYP/test_videos/cat_dog.mp4: 384x640 Done. (0.016s)
video 1/1 (13171/18163) /content/drive/.shortcut-targets-by-id/1WsrqcNndN0mishZGJ-SYTtzbRoQ2BzOQ/FYP/test_videos/cat_dog.mp4: 384x640 Done. (0.016s)
video 1/1 (13172/18163) /content/drive/.shortcut-target

In [ ]:
!python detect.py --weights /content/drive/MyDrive/FYP/exp/content/yolov5/runs/train/exp3/weights/best.pt --img 640 --conf 0.5 --source /content/drive/MyDrive/FYP/test_videos/cat_dog.mp4

detect: weights=['/content/drive/MyDrive/FYP/exp/content/yolov5/runs/train/exp3/weights/best.pt'], source=/content/drive/MyDrive/FYP/test_videos/cat_dog.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-344-g0e165c5 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 20865057 parameters, 0 gradients, 47.9 GFLOPs
video 1/1 (1/18163) /content/drive/.shortcut-targets-by-id/1WsrqcNndN0mishZGJ-SYTtzbRoQ2BzOQ/FYP/test_videos/cat_dog.mp4: 384x640 1 cat, Done. (0.022s)
video 1/1 (2/18163) /content/drive/.shortcut-targets-by-id/1WsrqcNndN0mishZGJ-SYTtzbRoQ2BzOQ/FYP/test_videos/cat_dog.mp4: 384x64